## Exercise 5.1

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.stats as st




def Exercise_5_1():
#     K = 10 #[109.90389652532572, 111.92839282255922]
    K = 100 #[111.82372311395758, 112.45993339317337]
    M = 100
    z_K = []
    np.random.seed(2)
    confidence = 0.95
    alpha = 1-confidence
    z = st.norm.ppf(1-alpha/2)

    
    for m in range(M):
        q_O = 1.2
        q_B = np.random.uniform(1.5,2.2,K)
        d = np.random.uniform(80,120,K)
        m = gp.Model("5_1")
        x_SH = m.addVar(vtype = GRB.CONTINUOUS, name = "the amount shipped from the factory")
        y_B = m.addVars([i for i in range(K)], vtype = GRB.CONTINUOUS, name = "the amount bought in the local market ")
        y_O = m.addVars([i for i in range(K)], vtype = GRB.CONTINUOUS, name = "the oversupply")
        
        m.setObjective(x_SH + (1/K)*sum(q_B[k]*y_B[k]+q_O*y_O[k] for k in range(K)),GRB.MINIMIZE)
        
        m.addConstr(x_SH <= 100)
        
        
        for k in range(K):
            m.addConstr(y_B[k]-y_O[k]+x_SH == d[k])
  
        
        m.optimize()
        print('x_SH:',x_SH.x)

        print('y_B:')
        for i in range(K):
            print(y_B[i].x)

        print('y_O:')
        for i in range(K):
            print(y_O[i].x)

        
        z_K.append(m.getObjective().getValue())
    
    
    
    L_MK = sum(z_K)/M
    
    
    #standard deviation
    theta = (1/(M-1)*sum((z_K[i]-L_MK)**2 for i in range(M)))**(1/2)
    
    
    cinterval = [L_MK-theta*z/(M**(1/2)),L_MK+theta*z/(M**(1/2))]
        
    print('z_K',z_K)
    print('L_MK',L_MK)
    print('standard deviation',theta)
    print('confidence interval', cinterval)

        
Exercise_5_1()